## script for combining each cluster assignment and metadata into one file for feeding into MASCOT GLM 
### needs to be repeated for each variant build (Alpha, Delta, Omicron, Other)

In [ ]:
import pandas as pd
import dendropy
import numpy as np
import re
import random

In [ ]:
#this step is currently hardcoded in, where you need to manually change the file path for each variant build
# will work on automating it soon
newickpath = "../../nextstrain_build/results/other_tree.nwk"
clusters = "../../nextstrain_build/cluster_assignment/other_new_cluster_assignment.tsv"
metadata = "../../nextstrain_build/results/other_sub_subsampled_metadata.tsv"


In [ ]:
#read in all three files
tree2 = dendropy.Tree.get(file=open(newickpath, "r"), schema="newick")
df = pd.read_csv(clusters, sep ="\t", index_col = 'strain')
meta_df = pd.read_csv(metadata, sep ="\t", index_col = 'strain')


In [ ]:
meta_df['ns_kc'] = meta_df['ns_kc'].replace("other_King County","Other_King_County")
df["location"] = np.nan
df['date'] = np.nan


In [ ]:
#for each leaf in the tree, add metadata information into cluster dataset. 
for leaf in tree2.leaf_node_iter():
    taxon_number = re.findall(r"'(.*?)'", str(leaf.taxon), re.DOTALL)
    if taxon_number[0] in list(df.index):
        df.loc[taxon_number[0], 'date'] =  meta_df.loc[taxon_number[0], 'date']
        df.loc[taxon_number[0], 'location'] =  meta_df.loc[taxon_number[0], 'ns_kc']


In [ ]:
#removing blanks and NAs for dates
df.replace("", np.nan, inplace=True)
df.dropna(subset = ["date"], inplace=True)

In [ ]:
#removing those for which we don't have north or south regional information for
df = df[df['location'] != 'Other_King_County']

In [ ]:
df

In [ ]:
#make sure to change the name for each respective variant build. Will automate soon.
df.to_csv('../data/kc_clusters_other_new.tsv', sep = "\t")